In [1]:
# MODEL = "llama3.2:latest"
MODEL = "mistral:latest"
CHROMA_PATH = "chroma"
DATA_PATH = "mydocs"


In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    print("Loading documents")
    loader = PyPDFDirectoryLoader(DATA_PATH)
    documents = loader.load()
    return documents

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_text(documents: list[Document]):
    print("Splitting text")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import os
import shutil

def save_to_chroma(chunks: list[Document]):
    print("Saving to Chroma")
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OllamaEmbeddings(model="nomic-embed-text"), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [6]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)

In [7]:
generate_data_store()

Loading documents
Splitting text
Split 23 documents into 136 chunks.
 Obvious: Use visual cues, like placing a book on your pillow to read before bed 
 Attractive: Pair an enjoyable activity with a necessary one (e.g., listening to podcasts 
while exercising) 
 Easy: Reduce steps needed to start a habit (e.g., sleeping in workout clothes) 
 Satisfying: Celebrate small wins and track progress visually
{'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-02-23T11:10:24-05:00', 'author': '', 'moddate': '2025-02-23T11:10:24-05:00', 'title': 'Microsoft Word - Atomic Habits', 'source': 'mydocs\\Atomic Habits.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2', 'start_index': 797}
Saving to Chroma


C:\Users\ipriv\AppData\Local\Temp\ipykernel_8940\4200232466.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  chunks, OllamaEmbeddings(model="nomic-embed-text"), persist_directory=CHROMA_PATH


Saved 136 chunks to chroma.


C:\Users\ipriv\AppData\Local\Temp\ipykernel_8940\4200232466.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [5]:
from langchain_community.llms import Ollama
model = Ollama(model= MODEL)

query_text = "What is framing?"

# Prepare the DB.
embedding_function = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=4)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

results

C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\3395372694.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model= MODEL)
C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\3395372694.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function = OllamaEmbeddings(model="nomic-embed-text")
C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\3395372694.py:8:

Unable to find matching results.


C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\3395372694.py:11: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'author': '', 'creationdate': '2025-02-23T11:11:14-05:00', 'creator': 'PyPDF', 'moddate': '2025-02-23T11:11:14-05:00', 'page': 5, 'page_label': '6', 'producer': 'Microsoft: Print To PDF', 'source': 'mydocs\\Thinking fast and slow.pdf', 'start_index': 0, 'title': 'Microsoft Word - Thinking fast and slow', 'total_pages': 8}, page_content='"The statement of a problem guides the selection of the relevant precedent, and the \nprecedent in turn frames the problem and thereby biases the solution. " \nFraming eƯects. The way information is presented (framed) can signiﬁcantly inﬂuence \ndecision-making, even when the underlying facts remain the same. This eƯect'), -280.8838222180336), (Document(metadata={'author': '', 'creationdate': '2025-02-23T11:11:14-05:00', 'creator': 'PyPDF', 'moddate': '2025-02-23T11:11:14-05:00', 'page': 5, 'page_label': '6'

[(Document(metadata={'author': '', 'creationdate': '2025-02-23T11:11:14-05:00', 'creator': 'PyPDF', 'moddate': '2025-02-23T11:11:14-05:00', 'page': 5, 'page_label': '6', 'producer': 'Microsoft: Print To PDF', 'source': 'mydocs\\Thinking fast and slow.pdf', 'start_index': 0, 'title': 'Microsoft Word - Thinking fast and slow', 'total_pages': 8}, page_content='"The statement of a problem guides the selection of the relevant precedent, and the \nprecedent in turn frames the problem and thereby biases the solution. " \nFraming eƯects. The way information is presented (framed) can signiﬁcantly inﬂuence \ndecision-making, even when the underlying facts remain the same. This eƯect'),
  -280.8838222180336),
 (Document(metadata={'author': '', 'creationdate': '2025-02-23T11:11:14-05:00', 'creator': 'PyPDF', 'moddate': '2025-02-23T11:11:14-05:00', 'page': 5, 'page_label': '6', 'producer': 'Microsoft: Print To PDF', 'source': 'mydocs\\Thinking fast and slow.pdf', 'start_index': 243, 'title': 'Micro

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [7]:
from langchain.prompts import ChatPromptTemplate

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Human: 
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: "The statement of a problem guides the selection of the relevant precedent, and the 
precedent in turn frames the problem and thereby biases the solution. " 
Framing eƯects. The way information is presented (framed) can signiﬁcantly inﬂuence 
decision-making, even when the underlying facts remain the same. This eƯect

---

decision-making, even when the underlying facts remain the same. This eƯect 
demonstrates that our preferences are not as stable as we might think and are often 
constructed in the moment based on context. 
Types of framing. Common framing eƯects include: 
 Gain vs. loss framing (e.g., "90% survival rate" vs. "10% mortality rate")

---

 Gain vs. loss framing (e.g., "90% survival rate" vs. "10% mortality rate") 
 Positive vs. negative framing (e.g., "95% fat-free" vs. "5% fat") 
 Temporal framing (e.g., short-term vs. long-term consequences) 

C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\993480867.py:8: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Response:  Framing refers to the way information is presented, which can significantly influence decision-making even when the underlying facts remain the same. Common types of framing include gain vs. loss framing, positive vs. negative framing, and temporal framing. Understanding these biases can help individuals and organizations make more rational decisions in various areas such as negotiations, investments, product pricing strategies, marketing and advertising, public policy communication, medical decision-making, and financial choices. It also impacts emotional attachment, sense of ownership, reference points, and expectations.
Sources: ['mydocs\\Thinking fast and slow.pdf', 'mydocs\\Thinking fast and slow.pdf', 'mydocs\\Thinking fast and slow.pdf', 'mydocs\\Thinking fast and slow.pdf']


In [9]:
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = OllamaEmbeddings(model="nomic-embed-text")
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(template)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)

    # sources = [doc.metadata.get("id", None) for doc, _score in results]
    # formatted_response = f"Response: {response_text}\nSources: {sources}"
    # print(formatted_response)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    
    return response_text

In [10]:
query_rag("How do i form good habits?")

Response:  To form good habits, follow these steps:

1. Identify a current habit you already do each day and stack your new behavior on top using the technique called Habit Stacking.
2. Increase friction for bad habits (e.g., unplug the TV after each use).
3. Decrease friction for good habits (e.g., prepare gym clothes the night before).
4. Make habits obvious, attractive, easy, and satisfying by following the Four Laws of Behavior Change:
   a. Make it obvious: Increase visibility of cues that trigger desired habits.
   b. Make it attractive: Associate habits with positive emotions or rewards.
5. Create chains of habits by anchoring new habits to existing ones, and chaining small habits together using the natural momentum that comes from one behavior leading into the next.
6. Focus on mastering the art of showing up and making habits automatic by spending the first two minutes on your desired habit to lower the barrier to entry and increase the chances of long-term success.
7. Change 

" To form good habits, follow these steps:\n\n1. Identify a current habit you already do each day and stack your new behavior on top using the technique called Habit Stacking.\n2. Increase friction for bad habits (e.g., unplug the TV after each use).\n3. Decrease friction for good habits (e.g., prepare gym clothes the night before).\n4. Make habits obvious, attractive, easy, and satisfying by following the Four Laws of Behavior Change:\n   a. Make it obvious: Increase visibility of cues that trigger desired habits.\n   b. Make it attractive: Associate habits with positive emotions or rewards.\n5. Create chains of habits by anchoring new habits to existing ones, and chaining small habits together using the natural momentum that comes from one behavior leading into the next.\n6. Focus on mastering the art of showing up and making habits automatic by spending the first two minutes on your desired habit to lower the barrier to entry and increase the chances of long-term success.\n7. Change

In [ ]:
def chatbot_simulation():
    print("Hello! I'm your chatbot. Ask me anything.")
    
    while True:
        # Ask for the user's query
        user_input = input("Your question: ")
        
        # If the user types 'exit', quit the loop
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye! Have a great day!")
            break
        
        # Rephrase the user's query using Ollama
        rephrased_query = rephrase_query_with_langchain(user_input)
        
        # Print the rephrased query (optional, for transparency)
        print(f"Rephrased query: '{rephrased_query}'")
        
        # Query the RAG system with the rephrased query
        answer = query_rag(rephrased_query)
        
        # Display the answer
        print(f"\n{answer}")
        
        print("\nFeel free to ask another question or type 'exit' to quit.\n")

# Run the chatbot simulation with Ollama rephrasing
chatbot_simulation()

Hello! I'm your chatbot. Ask me anything.


NameError: name 'rephrase_query_with_ollama' is not defined

In [14]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define a prompt template for rephrasing
rephrase_prompt = PromptTemplate(
    input_variables=["query"],
    template="Rephrase the following question: '{query}'"
)

# Set up the LLM (Ollama) integration via Langchain
ollama_llm = Ollama(model="llama2")  # Replace with the correct model name if needed

# Create an LLMChain using the rephrase template and Ollama model
rephrase_chain = LLMChain(llm=ollama_llm, prompt=rephrase_prompt)

def rephrase_query_with_langchain(query: str) -> str:
    """
    Rephrase the user's query using Langchain and Ollama LLM.
    """
    # Use Langchain's LLMChain to rephrase the query
    rephrased_query = rephrase_chain.run(query)
    return rephrased_query


C:\Users\ipriv\AppData\Local\Temp\ipykernel_26780\1473813795.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rephrase_chain = LLMChain(llm=ollama_llm, prompt=rephrase_prompt)
